In [ ]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests

# Environment/configuration (override via env vars)
API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")
MODEL    = os.getenv("MODEL_NAME", "bens_model")

## API helper: `call_model_chat_completions`

This function calls an OpenAI-style `/v1/chat/completions` endpoint and returns a structured result. It is safe to run when the ASU API is reachable (on-campus or via VPN) and the `API_BASE`/`API_KEY` environment variables are set.

In [ ]:
def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.0,
                                timeout: int = 60) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": 128,
    }

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}

## Notes and safety

- Ensure you are on the ASU network or connected to VPN so `API_BASE` is reachable.
- Set `OPENAI_API_KEY` / `API_BASE` / `MODEL_NAME` in the environment before running API calls.
- The helper returns a dict with `ok` and `text`; inspect `raw` for the full response.

In [ ]:
# Demo: guarded example (won't run unless you set USE_DEMO = True)
USE_DEMO = False
if USE_DEMO:
    resp = call_model_chat_completions("Write a one-line greeting.")
    print('OK:', resp.get('ok'))
    print('Text:', resp.get('text'))

## Next steps

- Refactor this notebook by adding an `Agent` class cell and a `run()` function to replace the `if __name__ == '__main__'` script logic in `agent.py`.
- Integrate with `generate_answer_template.py` by importing the refactored functions or exporting the notebook back to `agent_converted.py`.
- Add tests and small unit checks in additional cells for regression assurance.